# WRF

## Compiling

### WRF on gadi: https://github.com/coecms/WRF

#### Install

    $ git ls-remote --heads https://github.com/coecms/WRF.git
    $ git clone -b V4.5.2 --recursive https://github.com/coecms/WRF.git

#### Build WRF

    $ cd WRF/WRF/
    $ ./run_compile -h
    $ ./run_compile
    $ ls -l main/*.exe


#### Build WPS

    $ cd WRF/WPS/
    $ ./run_compile -h
    $ ./run_compile
    $ ls -l *.exe
    $ ls -l */src/*.exe

## Configurations

### Input data

#### WPS V4 Geographical Static Data Downloads Page: https://www2.mmm.ucar.edu/wrf/users/download/get_sources_wps_geog.html

    $ tar xzvf geog_high_res_mandatory.tar.gz

WRF/WPS/namelist.wps &geogrid:
    
    geog_data_path = '/home/563/qg8515/data/sim/wrf/input/WPS_GEOG/'

## Running

### WPS

    $ cd WRF/WPS

#### geogrid

change namelist.wps based on an example from Wenhui

More information: https://ral.ucar.edu/sites/default/files/public/Lesson-wps.html

How to create the domain: Nesting in WRF https://silo.tips/download/dave-gill-matthew-pyle-nesting-in-wrf

    $ ./geogrid.exe

    # output geo_em_d0X.nc

#### ungrib

Initialize WRF with ERA5: https://dreambooker.site/2019/10/03/initializing-the-wrf-model-with-era5-pressure-level/

    $ ./link_grib.csh /home/563/qg8515/data/sim/wrf/input/era5/*

    $ ln -s ./ungrib/Variable_Tables/Vtable.ERA-interim.pl Vtable

    $ ./ungrib.exe

    # output FILE:XXX

#### metgrid

Download QNWFA_QNIFA_QNBCA_SIGMA_MONTHLY.dat from https://forum.mmm.ucar.edu/threads/thomson-aerosol-aware-mp-28-and-real-data-for-aerosols.11631/

Download QNWFA_QNIFA_SIGMA_MONTHLY.dat from https://www2.mmm.ucar.edu/wrf/users/physics/mp28_updated.html

    $ ln -s ./metgrid/METGRID.TBL.ARW METGRID.TBL

    $ ./metgrid.exe

    # output met_em.***

### WRF

#### namelist.input

Reference: https://github.com/wrf-model/WRF/blob/master/run/README.namelist
    https://www2.mmm.ucar.edu/wrf/users/wrf_users_guide/build/html/namelist_variables.html#
    https://www2.mmm.ucar.edu/wrf/users/docs/user_guide_v4/v4.2/users_guide_chap5.html
    https://www2.mmm.ucar.edu/wrf/users/namelist_best_prac_wrf.html


Best practices:



#### real

Link WPS output file to the run directory

    $ ln -sf /home/563/qg8515/code/WRF/WPS/met_em.d* ./

    $ qsub run_real

#### wrf

    $ qsub run_mpi

## Idealised

### compile

    $ cd WRF/WRF/
    $ ./run_compile -h
    $ ./run_compile -a 75 -n 0 --compile_case em_les -c
    $ mv main/*.exe test/em_les

### run

    $ cd test/em_les
    $ mpirun -np 1 ./ideal.exe
    $ mpirun -np 8 ./wrf.exe

## setup_wrf_nci

    submit_setup.sh
    config.json
    namelist.wrf

change when necessary

    main_script_template.sh
    run_script_template.sh
    cleanup_script_template.sh
    nccopy_compress_output.sh

    add_remove_var.txt
    nccopy_compress_output.sh

no need to change

    setup_for_wrf.py


# ACCESS-rAM

## Run ACCESS-rAM3

### Useful website

Run ACCESS-rAM3: https://docs.access-hive.org.au/models/run-a-model/run-access-ram/

RAL3 description: https://code.metoffice.gov.uk/trac/rmed/wiki/RAL3

RAL3 settings: https://code.metoffice.gov.uk/trac/roses-u/browser/b/y/3/9/5/trunk/app/um/rose-app.conf

Rosie: https://metomi.github.io/rose/doc/html/tutorial/rose/furthertopics/rosie

UM: https://code.metoffice.gov.uk/trac/um

### Set environment

Set up persistent session

In [ ]:
! persistent-sessions start -p gx60 <name> # Start a new persistent session
! echo "<name>.${USER}.gx60.ps.gadi.nci.org.au" > ~/.persistent-sessions/cylc-session       # Specify target persistent session

#---- other commands:
! persistent-sessions kill <persistent-session-uuid>
! persistent-sessions list

Rose/Cylc/MOSRS setup

In [ ]:
! module use /g/data/hr22/modulefiles
! module load cylc7

MOSRS authentication

In [ ]:
! mosrs-auth

### Regional Ancillary Suite (RAS)

Get the RAS configuration

In [ ]:
! rosie checkout u-dg767/access_rel_1       # Local-only copy
! rosie copy u-dg767/access_rel_1           # Remote and local copy

#---- other commands
! rosie help

Run the RAS

In [ ]:
! rose suite-run                # from within the RAS suite directory in roses/

#---- other commands
! cylc scan                     # scan for active suites
! rose suite-gcontrol           # reopen the Cylc GUI
! rose suite-stop -y            # shutdown a suite in a safe manner
! qstat -u $USER                # Check the status of all your PBS jobs
! qdel <job-ID>                 # Delete any job related to your run

! rose suite-run --restart      # SOFT restart, reinstall the suite and reopen Cylc in the same state it was prior to being stopped
! rose suite-run --new          # HARD restart, overwrite any previous runs of the suite and start afresh, a new submission of an experiment
! rose suite-run --reload       # RELOAD a suite
! rose suite-clean              # CLEAN a suite
! cylc ping -v --host=test.qg8515.gx60.ps.gadi.nci.org.au u-dg768

### Regional Nesting Suite (RNS)

Get the RNS configuration

In [ ]:
! rosie checkout u-dg768/access_rel_1

Run the RNS

In [ ]:
! rose suite-run

### Edit ACCESS-rAM3 configuration

#### Rose GUI

In [ ]:
! rose edit &               # from within the suite directory

#### Change run length

**RNS**: `INITIAL_CYCLE_POINT`, `FINAL_CYCLE_POINT` (+P2D-PT1S): *suite conf → Nesting Suite → Cycling*

#### Change the land-surface initial conditions source

**RNS**: `NCI_HRES_ECCB`: *suite conf → Nesting Suite → Driving model setup*

#### Change the simulation region (both RAS and RNS)

##### Change the nested region name

**RAS**: `rg01_name`: *suite conf → Regional Ancillary Suite → Nested region 1 setup*

**RNS**:

1. `rg01_rs01_ancil_dir`: *suite conf → Nesting Suite → Nested region 1 setup → Resolution 1 setup*

2. `rg01_rs02_ancil_dir`: *suite conf → Nesting Suite → Nested region 1 setup → Resolution 2 setup*

3. `dm_ec_lam_ancil_dir`: *suite conf → Nesting Suite → Driving model setup*

4. `rg01_name`: *suite conf → Nesting Suite → Nested region 1 setup*


##### Change the nested region position

**RAS**: `rg01_centre`: *suite conf → Regional Ancillary Suite → Nested region 1 setup*

##### Change the nested region's nest configuration

#### Change the output variables

**RNS**:

1. `rg01_rs01_m01_stashpack`: *suite conf → Nesting Suite → Nested region 1 setup → Resolution 1 setup → Config 1 setup*

2. `rg01_rs02_m02_stashpack`: *suite conf → Nesting Suite → Nested region 1 setup → Resolution 2 setup → Config 2 setup*

## FCM

### Basic command

In [ ]:
%% bash

fcm diff
fcm status
fcm checkout
fcm commit
fcm branch
fcm merge
fcm help <sub-command>

# Create a branch
fcm branch-create -k <ticket> <branch_name> fcm:um.x-tr@vn11.8
fcm branch-create --help
fcm checkout URL
fcm add --help
fcm delete --help
fcm revert  --help
fcm branch-info


# ACCESS-AM3

## Instructions to run ACCESS-AM3

### Links

Run ACCESS-AM3: https://docs.access-hive.org.au/pr-previews/1058/models/run_a_model/run_access-am3/


Others:

UM: https://github.com/ACCESS-NRI/UM

JULES: https://github.com/ACCESS-NRI/JULES

GCOM4: https://github.com/ACCESS-NRI/GCOM4

ACCESS-AM3: https://github.com/ACCESS-NRI/ACCESS-AM3

Model Live Diagnostics: https://med-live-diagnostics.readthedocs.io/en/latest/index.html

ACCESS-CM2 and ESM1.5 data archive p73: https://research.csiro.au/access/wiki/access-model-output-archive-p73-wiki/

### Instructions to run ACCESS-AM3

#### Get the configuration

<!-- Step 1: fork https://github.com/ACCESS-NRI/access-am3-configs -->

    git clone https://github.com/l975421700/access-am3-configs.git
    git checkout dev-n96e
    git checkout -b n96e-vn13.1
    git push -u origin n96e-vn13.1

<!-- $ git clone https://github.com/ACCESS-NRI/access3-configs.git -->

Useful git command:

    git stash
    git pull
    git stash pop

    git rev-parse --abbrev-ref HEAD

Initial modification

#### Set your NCI projects

change root_dir, STORAGE_PROJECT, and COMPUTE_PROJECT in rose_suite.conf_nci_gadi

change storage in site/nci_gadi.rc